<img width="1347" height="392" alt="image" src="https://github.com/user-attachments/assets/df0f71b5-7bae-4751-a85c-92f9cfbf61af" />
<a href="https://github.com/WilmaDarc/volta-zero-/tree/main">

---

<!-- # **Profissão: Cientista de Dados** -->
### **Módulo 23** | Combinação de modelos I | Exercício 2

**Aluno:** [Wilma Darc Alves de Farias](www.linkedin.com/in/wilma-farias-66a15962)<br>
**Data:** 21 de dezembro de 2025.

---

# Tarefa 02

## **1.** Monte um passo a passo para o algoritmo RF

> Semelhante ao *Bagging*, o ***Random Forest*** consiste nos seguintes passos:
> 1. ***Bootstrap + Feature Selection:***:
>   
>    1.1. Amostragem Bootstrap (Linhas):Assim como no Bagging tradicional, o Random Forest começa criando vários subconjuntos de dados. Para cada árvore da "floresta", o algoritmo sorteia registros do banco de dados original com reposição.Objetivo: Garantir que cada árvore veja uma combinação de dados levemente diferente.
>
>    1.2.Seleção Aleatória de Recursos (Colunas):Este é o "pulo do gato" que diferencia o Random Forest. Em cada nó de cada árvore, o algoritmo não olha para todas as variáveis (features) disponíveis. Ele sorteia apenas um subconjunto aleatório de colunas (geralmente a raiz quadrada do total de colunas).Por que isso ocorre? Para evitar que uma variável muito dominante (como "salário" em um modelo de crédito) dite o comportamento de todas as árvores, permitindo que variáveis menos óbvias também sejam testadas.
> 
> 3. **Modelagem com árvores de decisão:** Nesta etapa, um modelo de *Machine Learning*, especificamente uma árvore de decisão, é treinado de forma independente em cada amostra *bootstrap* com as variáveis aleatórias que foram definidas no passo anterior.
> 4. **Agregação:** Por fim, os resultados de cada modelo independente (cada árvore de decisão) são agregados para obter uma previsão final. Em problemas de classificação, a agregação é geralmente feita por meio do voto majoritário, em que a classe prevista com mais frequência pelos modelos individuais é selecionada como a classe final. Já em problemas de regressão, a agregação é realizada calculando a média das previsões dos modelos individuais.

## **2.** Explique com suas palavras o Random Forest

> ***Random Forest*** pode ser definido como uma extensão do *Bootstrap Aggregating*, conhecido como *Bagging*, que oferece melhorias em desempenho e resultados. Assim como no *Bagging*, o *Random Forest* é um método de combinação de modelos de *Machine Learning*, em que cada modelo é treinado em variações do conjunto de dados original. Essas variações, chamadas de *amostras bootstrap*, consistem em subconjuntos aleatórios do conjunto de dados original, com possíveis repetições, mantendo o mesmo número de linhas.
>
> Entretanto, o *Random Forest* diferencia-se ao utilizar apenas uma quantidade determinada e aleatória de variáveis em cada modelo, especificamente em modelos de árvore de decisão. Essa abordagem tem o objetivo de reduzir a variância dos resultados e atenuar o risco de *overfitting*. Para obter a previsão final, a agregação dos modelos é realizada de forma semelhante ao *Bagging*, onde a média é utilizada para regressão e votação para classificação.
>
 >**Vantagens do Random Forest**:
>
>Robustez: É muito difícil de sofrer overfitting.
>
>Importância das Variáveis: Ele consegue dizer quais colunas foram mais importantes para a decisão final (atributo feature_importances_).
>
>Versatilidade: Funciona bem tanto para dados numéricos quanto categóricos com pouco pré-processamento. 





## **3.** Qual a diferença entre Bagging e Random Forest?

> A diferença entre *Bagging* e *Random Forest* está no desempenho e na redução da variância entre os resultados dos modelos. *Bagging* é uma técnica mais geral de combinação de modelos de *Machine Learning* utilizando amostragem aleatória, enquanto *Random Forest* é uma variação mais específica desse método que utiliza árvores de decisão. No *Random Forest*, cada modelo é treinado em subconjuntos aleatórios do conjunto de dados original, com uma quantidade reduzida e aleatória de variáveis consideradas em cada modelo. Essa abordagem aumenta a robustez do modelo e, ao combinar os resultados dos modelos, a previsão final tende a ser mais precisa do que no *Bagging*. Portanto, *Random Forest* é uma extensão aprimorada do *Bagging*, fornecendo um desempenho melhorado e uma redução adicional na variância dos resultados.
>
>> **"*Random Forest* funciona melhor que o *Bagging*, pois as árvores amostradas são mais independentes (menor correlação)."**

## **4.** (Opcional) Implementar em python o Random Forest
> - Bootstrap
> - Feature selection
> - Modelagem com Decision trees
> - Agregação

In [1]:
# Import das bibliotecas:

import numpy  as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.datasets import load_diabetes

from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
# Exemplo da técnica Random Forest para problemas de classificação:

X = load_iris().data
y = load_iris().target

df = pd.DataFrame(X, columns=load_iris().feature_names)
df['target'] = y

def rf_classifier(df:pd.DataFrame, 
                  num_bootstrap_samples:int=3,  # Parâmetro da função que define a quantidade de amostragens para treinamento
                  test_size:float=0.25
                 ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)  # Amostragem COM reposição

        X_train = df_train.drop(['target'], axis=1)
        # Feature selection
        X_train = X_train.sample(n=round(np.sqrt(X_train.shape[1])),  # Cálculo da raiz quadrada da quantidade de variáveis
                                 axis=1)
        
        y_train = df_train['target']
        
        # Modelagem (base learners)
        model = DecisionTreeClassifier()
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao dicionário para agregação das predições
        y_pred_bagging.update({i:model.predict(X_test[X_train.columns])})
    
    # Aggregating
    y_pred = (pd.DataFrame(y_pred_bagging)
                .mode(axis=1)  # Agregando o valor com maior número de aparições nas predições dos modelos
                .rename(columns={0:'y_pred'}))
 
    # Resultados
    print(model)
    print('Accuracy score:', accuracy_score(y_true=y_test, 
                                            y_pred=y_pred['y_pred']
                                           ))

    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred['y_pred'].astype(int)], 
                     axis=1)

rf_classifier(num_bootstrap_samples=10, df=df, test_size=0.33)

DecisionTreeClassifier()
Accuracy score: 0.96


,y_test,y_pred
0,0,0
1,0,0
2,1,1
3,0,0
4,0,0
5,0,0
6,2,1
7,1,1
8,1,1
9,2,2


In [3]:
# Exemplo da técnica Random Forest para problemas de regressão:

X = load_diabetes().data
y = load_diabetes().target

df = pd.DataFrame(X, columns=load_diabetes().feature_names)
df['target'] = y

def rf_regressor(df:pd.DataFrame, 
                 num_bootstrap_samples:int=3,  # Parâmetro da função que define a quantidade de amostragens para treinamento
                 test_size:float=0.25
                ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)  # Amostragem COM reposição

        X_train = df_train.drop(['target'], axis=1)
        # Feature selection
        X_train = X_train.sample(n=round(X_train.shape[1]/3),  # Cálculo da quantidade de variáveis dividida por 3
                                 axis=1)
        
        y_train = df_train['target']

        # Modelagem (base learners)
        model = DecisionTreeRegressor()
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao dicionário para agregação das predições
        y_pred_bagging.update({i:model.predict(X_test[X_train.columns])})

    # Aggregating
    y_pred = (pd.DataFrame(y_pred_bagging)
                .mean(axis=1)  # Agregando as predições dos modelos baseando n a média dos resultados
                .rename('y_pred'))
 
    # Resultados
    print(model)
    print('Mean squared error:', mean_squared_error(y_true=y_test, 
                                                   y_pred=y_pred))
    print('Coefficient of determination:', r2_score(y_true=y_test, 
                                                    y_pred=y_pred))
    
    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred], 
                     axis=1)
    
rf_regressor(num_bootstrap_samples=10, df=df, test_size=0.33)

DecisionTreeRegressor()
Mean squared error: 5943.812271068245
Coefficient of determination: 0.06616946929456136


,y_test,y_pred
0,142.0,128.8
1,64.0,124.6
2,158.0,161.9
3,55.0,163.7
4,219.0,166.9
...,...,...
141,48.0,172.3
142,173.0,216.5
143,74.0,187.0
144,243.0,219.5
